In [1]:
import pandas as pd
import numpy as np

In [31]:
rent_df = pd.read_csv("./서울시_임대료_데이터.csv", encoding="CP949", skiprows = 1)
rent_df = rent_df.drop("Unnamed: 0", axis = 1)
rent_df = rent_df.rename(columns = {"Unnamed: 1" : "지역"})
rent_df = rent_df.set_index("지역")
rent_df.columns = [["중대형", "중대형", "중대형", "중대형", "중대형", "중대형", "중대형", "소형", "소형", "소형"],
                 ["지하1층", "1층", "2층", "3층", "4층", "5층", "6-10층", "지하1층", "1층", "2층"]]
rent_df.head()

중대형                                         소형             
     지하1층     1층    2층    3층    4층    5층 6-10층  지하1층     1층    2층
지역                                                               
전체   15.4   52.5  23.9  19.0  17.7  17.8  17.4  23.4   49.4  25.7
광화문  28.1   80.7  41.1  30.1  24.3  23.4  23.8   NaN   83.2  41.8
남대문  19.6   69.0  24.6  19.9  18.2  15.3  16.9   NaN   64.3  24.1
동대문  13.8   45.6  18.7  15.0  14.4  17.1  19.9   8.5   48.0  21.9
명동   28.9  194.0  79.4  59.5  54.4  42.5  35.8  40.2  140.4  50.9

In [37]:
rent_df = rent_df.fillna(0)

In [38]:
rent_df.head()

중대형                                         소형             
     지하1층     1층    2층    3층    4층    5층 6-10층  지하1층     1층    2층
지역                                                               
전체   15.4   52.5  23.9  19.0  17.7  17.8  17.4  23.4   49.4  25.7
광화문  28.1   80.7  41.1  30.1  24.3  23.4  23.8   0.0   83.2  41.8
남대문  19.6   69.0  24.6  19.9  18.2  15.3  16.9   0.0   64.3  24.1
동대문  13.8   45.6  18.7  15.0  14.4  17.1  19.9   8.5   48.0  21.9
명동   28.9  194.0  79.4  59.5  54.4  42.5  35.8  40.2  140.4  50.9

In [40]:
rent_df.shape

(55, 10)

In [41]:
rent_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55 entries, 전체 to 화곡
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   (중대형, 지하1층)   55 non-null     float64
 1   (중대형, 1층)     55 non-null     float64
 2   (중대형, 2층)     55 non-null     float64
 3   (중대형, 3층)     55 non-null     float64
 4   (중대형, 4층)     55 non-null     float64
 5   (중대형, 5층)     55 non-null     float64
 6   (중대형, 6-10층)  55 non-null     float64
 7   (소형, 지하1층)    55 non-null     float64
 8   (소형, 1층)      55 non-null     float64
 9   (소형, 2층)      55 non-null     float64
dtypes: float64(10)
memory usage: 4.7+ KB


In [43]:
rent_df.describe()

중대형                                                          \
            지하1층          1층         2층         3층         4층         5층   
count  55.000000   55.000000  55.000000  55.000000  55.000000  55.000000   
mean   15.316364   52.650909  24.001818  18.810909  17.187273  16.305455   
std     5.770576   25.167975  10.257635   7.596482   7.193440   5.865859   
min     7.400000   27.100000  14.100000  10.600000   9.100000   9.100000   
25%    11.650000   38.550000  17.900000  14.500000  12.850000  12.200000   
50%    13.600000   47.200000  21.100000  16.800000  15.600000  15.200000   
75%    17.300000   59.950000  27.200000  20.900000  18.850000  18.000000   
max    34.400000  194.000000  79.400000  59.500000  54.400000  42.500000   

                         소형                         
           6-10층       지하1층          1층         2층  
count  55.000000  55.000000   55.000000  55.000000  
mean   14.632727  16.705455   50.354545  25.065455  
std     9.165132  13.316551   19.288404  10.254560  
min     0.000000   0.000000   26.000000   9.800000  
25%    10.850000   9.150000   37.300000  16.700000  
50%    16.000000  16.000000   46.700000  23.500000  
75%    20.650000  22.150000   58.650000  31.750000  
max    35.800000  54.100000  140.400000  52.400000

In [44]:
rent_df.dtypes

중대형  지하1층     float64
     1층       float64
     2층       float64
     3층       float64
     4층       float64
     5층       float64
     6-10층    float64
소형   지하1층     float64
     1층       float64
     2층       float64
dtype: object

In [51]:
emp_df = pd.read_excel("./Data/사업체+및+종사자+밀도(동별)_20240312165222.xlsx", skiprows = 1)
# emp_df = pd.read_excel("./Data/사업체+및+종사자+밀도(동별)_20240312165222.xlsx")
emp_df = emp_df.drop("동별(1)", axis = 1)
emp_df = emp_df.rename(columns = {"동별(2)" : "자치구"})
emp_df = emp_df.set_index("자치구")

level1_columns = []
for i in range(2016, 2022):
    for j in range(7):
        level1_columns.append(str(i))

level2_columns = []
for i in range(6):
    for j in ["사업체수", "종사자수", "평균종사자", "사업체밀도", "종사자밀도(A)", "인구밀도(B)", "종사자밀도비(A/B*100)"]:
        level2_columns.append(j)

emp_df.columns = [level1_columns, level2_columns]

emp_df.head()

C:\Users\lenovo\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


2016                                                        2017  \
      사업체수    종사자수 평균종사자 사업체밀도 종사자밀도(A) 인구밀도(B) 종사자밀도비(A/B*100)   사업체수   
자치구                                                                      
종로구  40664  268309  6.60  1701    11220    6771          165.70  40490   
중구   63640  402892  6.33  6389    40447   13494          299.75  62936   
용산구  20994  129033  6.15   960     5901   11209           52.64  20806   
성동구  26581  165047  6.21  1577     9789   18218           53.73  27278   
광진구  24895  126937  5.10  1459     7439   21807           34.11  24688   

                   ...      2020                             2021          \
       종사자수 평균종사자  ...  종사자밀도(A)   인구밀도(B) 종사자밀도비(A/B*100)   사업체수    종사자수   
자치구                ...                                                      
종로구  268702  6.64  ...  11900.14   6648.89          178.98  48361  275063   
중구   386648  6.14  ...  40969.03  13517.16          303.09  70308  386564   
용산구  133260  6.40  ...   7056.02  11188.30           63.07  29680  152605   
성동구  167308  6.13  ...  11646.82  17822.49           65.35  41665  203221   
광진구  125170  5.07  ...   7866.76  21104.75           37.27  33706  129707   

                                                        
    평균종사자    사업체밀도  종사자밀도(A)   인구밀도(B) 종사자밀도비(A/B*100)  
자치구                                                     
종로구  5.69  2022.38  11502.69   6431.21          178.86  
중구   5.50  7058.85  38810.63  13231.28          293.32  
용산구  5.14  1357.35   6979.05  10851.71           64.31  
성동구  4.88  2471.25  12053.51  17359.06           69.44  
광진구  3.85  1975.39   7601.65  20666.18           36.78  

[5 rows x 42 columns]